```cypher
MATCH (n) DETACH DELETE n;
```

```bash
docker-compose up -d --build
```

```bash
docker-compose down
```

```cypher
MATCH (n)
RETURN n
```

```cypher
SHOW INDEXES;
DROP INDEX vector;
```

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings 
warnings.filterwarnings('ignore')

from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain_community.graphs.neo4j_graph import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_core.documents.base import Document
from langchain_community.document_loaders import (
    TextLoader,
    UnstructuredFileLoader,
    JSONLoader,
)

from vn_langchainLGT import LLMGraphTransformer
from typing import Sequence

from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os

from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver
from openai import OpenAI
from langchain.chat_models import ChatOpenAI

import logging
from tqdm import tqdm
import json
import re
from tqdm import tqdm
from time import sleep
from datetime import datetime

from PyPDF2 import PdfReader
import pdfplumber
from docx import Document as DocxDocument
import docx
from markdown import markdown

import pickle
from typing import List, Dict, Any
from pathlib import Path

from dotenv import load_dotenv

if not load_dotenv():
    print("Warning: .env not correctly set up")
else:
    print("Proceed")

C:\Users\PC\AppData\Local\Temp\ipykernel_19304\1502321853.py:19: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from vn_langchainLGT import LLMGraphTransformer


Proceed


In [4]:
MODEL = "aya-23-8b@q8_0"
EMB_MODEL = "text-embedding-multilingual-e5-large-instruct"
CHECKPOINT_PATH = "vi_processing_checkpoint.json"
DOC_DIR = "tailieu/"
ALLOWED_NODES = []
ALLOWED_RELATIONSHIPS = []
STRICT_MODE = False

In [5]:
from neo4j import GraphDatabase

def is_database_populated(uri="bolt://localhost:7687", user="neo4j", password="your_password"):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) AS node_count")
        node_count = result.single()["node_count"]
        
    driver.close()
    return node_count > 0

if not is_database_populated():
    print("DB is not populated")
else:
    print("DB IS POPULATED")

DB is not populated


In [7]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

class CheckpointManager:
    def __init__(self, path: str = CHECKPOINT_PATH):
        self.path = path
        self.data: Dict[str, Any] = {
            "processed_files": [],
            "converted_chunks": [],
            "inserted_documents": [],  # Track inserted documents by doc_id
            "inserted_batches": [],  # Track batch indices for logging purposes
            "processed_graph_docs": [],  # New field to track processed graph docs by doc_id
        }
        if os.path.exists(path):
            self.load()

    def load(self):
        with open(self.path, "r") as f:
            self.data = json.load(f)

    def save(self):
        with open(self.path, "w") as f:
            json.dump(self.data, f)

    def is_file_processed(self, file_path: str, file_hash: int) -> bool:
        """Check if a file has been processed by its hash."""
        return file_hash in self.data["processed_files"]

    def add_processed_file(self, file_path: str, file_hash: int):
        """Mark a file as processed by its hash."""
        if file_hash not in self.data["processed_files"]:
            self.data["processed_files"].append(file_hash)
            self.save()

    def add_converted_chunk(self, chunk_hash: int):
        """Mark a chunk as converted by its hash."""
        if chunk_hash not in self.data["converted_chunks"]:
            self.data["converted_chunks"].append(chunk_hash)
            self.save()

    def is_graph_doc_processed(self, doc_id: str) -> bool:
        """Check if a graph document has been processed by its doc_id."""
        return doc_id in self.data["processed_graph_docs"]

    def add_processed_graph_doc(self, doc_id: str):
        """Mark a graph document as processed by its doc_id."""
        if doc_id not in self.data["processed_graph_docs"]:
            self.data["processed_graph_docs"].append(doc_id)
            self.save()

    def is_document_inserted(self, doc_hash: int) -> bool:
        """Check if a document has been inserted into Neo4j by its doc_hash."""
        return doc_hash in self.data["inserted_documents"]

    def add_inserted_document(self, doc_id: str):
        """Mark a document as inserted into Neo4j by its doc_id."""
        if doc_id not in self.data["inserted_documents"]:
            self.data["inserted_documents"].append(doc_id)
            self.save()

    def add_inserted_batch(self, batch_id: int):
        """Mark a batch as inserted."""
        if batch_id not in self.data["inserted_batches"]:
            self.data["inserted_batches"].append(batch_id)
            self.save()


def strip_markdown(text: str) -> str:
    """Simplify markdown text to plain text"""
    html = markdown(text)
    return re.sub(r'<.*?>', '', html)

def extract_json_strings(data: Any) -> str:
    """Recursively extract all strings from JSON data"""
    if isinstance(data, dict):
        return " ".join(extract_json_strings(v) for v in data.values())
    if isinstance(data, list):
        return " ".join(extract_json_strings(v) for v in data)
    return str(data) if isinstance(data, str) else ""

def extract_pdf_text(file_path: str) -> str:
    """Extract text from PDF using PyPDF2"""
    try:
        reader = PdfReader(file_path)
        pdf_text = []
        for page in reader.pages:
            text = page.extract_text()
            if text:
                pdf_text.append(text)
        return "\n".join(pdf_text)
    except Exception as e:
        logging.error(f"Error extracting PDF text from {file_path}: {e}")
        return ""

def extract_docx_text(file_path: str) -> str:
    """Extract text from DOCX file using python-docx"""
    try:
        doc = DocxDocument(file_path)
        doc_text = []
        for para in doc.paragraphs:
            doc_text.append(para.text)
        return "\n".join(doc_text)
    except Exception as e:
        logging.error(f"Error extracting DOCX text from {file_path}: {e}")
        return ""

def load_and_clean(file_path: str) -> str:
    """Load and clean file content based on file type"""
    ext = os.path.splitext(file_path)[1].lower()
    try:
        if ext == ".md":
            # Markdown files - convert to plain text
            with open(file_path, "r", encoding="utf-8") as f:
                markdown_content = f.read()
            return strip_markdown(markdown_content)
        elif ext == ".json":
            # JSON files - extract strings from the JSON object
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            return extract_json_strings(data)
        elif ext == ".pdf":
            # PDF files - extract text using PyPDF2
            return extract_pdf_text(file_path)
        elif ext == ".docx":
            # DOCX files - extract text using python-docx
            return extract_docx_text(file_path)
        elif ext == ".txt":
            # For text files, just load directly
            loader = TextLoader(file_path)
            return loader.load()[0].page_content
        else:
            logging.warning(f"Unsupported file type: {file_path}")
            return ""
    except Exception as e:
        logging.error(f"Error loading {file_path}: {e}")
        return ""

###### PIPELINE
checkpoint = CheckpointManager()
graph = Neo4jGraph()
client = ChatOpenAI(
    base_url="http://127.0.0.1:8000/v1",
    api_key="lm-studio",
    model=MODEL,
    temperature=0,
    request_timeout=240,
)

llm_transformer = LLMGraphTransformer(
    llm=client,
    allowed_nodes=ALLOWED_NODES,
    allowed_relationships=ALLOWED_RELATIONSHIPS,
    strict_mode=STRICT_MODE,
)

# File discovery and sorting
file_paths = sorted(
    [
        os.path.join(root, f)
        for root, _, files in os.walk(DOC_DIR)
        if ".ipynb_checkpoints" not in root
        for f in files
        if f.lower().endswith(("md", "json", "docx", "pdf", "txt"))
    ]
)

# Text processing pipeline
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=75,
    chunk_overlap=15,
    length_function=lambda x: len(x.split()),
    separators=[" "]  # Only split by spaces, no newlines
)

documents: List[Document] = []

# Stage 1: File loading and splitting
for file_path in tqdm(file_paths, desc="Processing files"):
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        content = f.read()
    file_hash = hash(content)

    # Check if the file has been processed already by checking the content hash
    if checkpoint.is_file_processed(file_path, file_hash):
        logging.info(f"File {file_path} has already been processed.")
        continue  # Skip if the file has already been processed

    try:
        # Load and clean file content
        content = load_and_clean(file_path)
        if not content:
            logging.warning(f"File {file_path} is empty after cleaning. Skipping.")
            continue

        # Split the document into chunks
        chunks = text_splitter.create_documents([content])

        # Add metadata (filename and any other relevant details)
        for chunk in chunks:
            chunk.metadata = {
                'source': file_path,  # Store the filename in the chunk metadata
                'hash': file_hash  # Store the hash of the file content for comparison
            }

        documents.extend(chunks)

        # Add file to checkpoint with content hash
        checkpoint.add_processed_file(file_path, file_hash)  # Ensure file_hash is stored
        logging.info(f"Processed {file_path} into {len(chunks)} chunks")

    except Exception as e:
        logging.error(f"Failed to process {file_path}: {e}")
        continue  # Skip this file and continue with others

# Stage 2: Graph document conversion
graph_documents = []
for doc in tqdm(documents, desc="Converting to graph"):
    doc_id = f"{doc.metadata.get('source', 'unknown')}_{hash(doc.page_content)}"

    # Check if this graph document has already been processed
    if checkpoint.is_graph_doc_processed(doc_id):
        logging.info(f"Graph document {doc_id} has already been processed.")
        continue

    max_retries = 5
    retry_delay = 1  # Seconds before retrying
    attempt = 0

    while attempt < max_retries:
        try:
            # Attempt to convert the document
            graph_doc = llm_transformer.convert_to_graph_documents([doc])[0]

            # Assign original metadata to GraphDocument
            graph_doc.source = doc
            graph_documents.append(graph_doc)

            # Mark this graph document as processed
            checkpoint.add_processed_graph_doc(doc_id)
            logging.info(f"Converted document {doc_id} to GraphDocument.")
            break  # Success, exit retry loop

        except Exception as e:
            attempt += 1
            logging.warning(f"Attempt {attempt}/{max_retries} failed for {doc_id}: {e}")

            if attempt < max_retries:
                sleep(retry_delay)  # Wait before retrying
            else:
                logging.error(f"Failed to convert document {doc_id} after {max_retries} attempts.")
                break  # Stop retrying after max attempts

# Stage 3: Add to Neo4j
batch_size = 10
for doc in tqdm(graph_documents, desc="Inserting documents"):
    doc_hash = hash(doc.source.page_content)

    if checkpoint.is_document_inserted(doc_hash):
        logging.info(f"Document with hash {doc_hash} has already been inserted.")
        continue

    try:
        source = doc.source.metadata.get('source', 'unknown')
        file_hash = doc.source.metadata.get('hash', doc_hash)
        doc_id = str(doc_hash)

        # Insert or update the Document node with page_content
        doc_query = f"""
        MERGE (doc:Document {{id: '{doc_id}'}})
        ON CREATE SET doc.source = '{source}', doc.hash = {file_hash}, doc.page_content = '{doc.source.page_content}'
        ON MATCH SET doc.page_content = '{doc.source.page_content}', doc.updated = timestamp()
        """
        graph.query(doc_query)

        for node in doc.nodes:
            # Apply only the __Entity__ label to all nodes
            node_query = f"""
            MERGE (n:__Entity__ {{id: '{node.id}'}})
            ON CREATE SET n.type = '{node.type}'
            """
            graph.query(node_query)
            
            # Linking Document to Node
            link_doc_node_query = f"""
            MATCH (doc:Document {{id: '{doc_id}'}})
            MATCH (n:__Entity__ {{id: '{node.id}'}})
            MERGE (doc)-[:Nhắc_đến]->(n)
            """
            graph.query(link_doc_node_query)
        
        for rel in doc.relationships:
            rel_query = f"""
            MERGE (source:__Entity__ {{id: '{rel.source.id}'}})
            MERGE (target:__Entity__ {{id: '{rel.target.id}'}})
            MERGE (source)-[r:{rel.type}]->(target)
            """
            graph.query(rel_query)

        checkpoint.add_inserted_document(doc_id)
        logging.info(f"Inserted document {doc_id}")

    except Exception as e:
        logging.error(f"Failed to insert document {doc_id}: {e}")
        continue

logging.info("Processing completed successfully")        

2025-03-19 23:15:50,368 - INFO - Processed tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md into 11 chunks00:00<?, ?it/s]
Processing files: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 123.39it/s]
2025-03-19 23:15:58,320 - INFO - HTTP Request: POST http://127.0.0.1:8000/v1/chat/completions "HTTP/1.1 200 OK", ?it/s]
2025-03-19 23:15:58,320 - INFO - Converted document tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md_-7808350393287397044 to GraphDocument.
2025-03-19 23:16:09,150 - INFO - HTTP Request: POST http://127.0.0.1:8000/v1/chat/completions "HTTP/1.1 200 OK".95s/it]
2025-03-19 23:16:09,150 - INFO - Converted document tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md_-2372475016998018864 to GraphDocument.
2025-03-19 23:16:22,929 - INFO - HTTP Request: POST http://127.0.0.1:8000/v1/chat/completions "HTTP/1.1 200 OK".64s/it]
2025-03-19 23:16:22,943 - INFO - Converted document tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md_-1153490466340006883 to G

In [8]:
import requests
import json

graph = Neo4jGraph()

def get_embedding(text):
    response = requests.post(
        "http://127.0.0.1:8000/v1/embeddings",
        headers={"Content-Type": "application/json"},
        json={"input": text, "model": EMB_MODEL}
    )
    return response.json()["data"][0]["embedding"]

# Custom embedding function
class LMStudioEmbeddings:
    def embed_documents(self, texts):
        return [get_embedding(text) for text in texts]
    
    def embed_query(self, text):
        return get_embedding(text)

# Use LM Studio for embeddings
embeddings = LMStudioEmbeddings()

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["page_content"],
    embedding_node_property="embedding"
)

vector_retriever = vector_index.as_retriever()

In [9]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
CREATE FULLTEXT INDEX `fulltext_entity_index` 
FOR (n:__Entity__) 
ON EACH [n.id, n.type];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [10]:
# Define the Entities model expecting 'names' field
class Entities(BaseModel):
    names: list[str] = Field(..., description="Extracted entities from the text.")

# Setup LLM model
llm = ChatOpenAI(
    openai_api_base="http://127.0.0.1:8000/v1",
    openai_api_key="lm-studio",
    model=MODEL,
    temperature=0
)

# Define the prompt template
prompt = ChatPromptTemplate.from_template(
"""
Bạn hãy trích xuất tất cả các tên người, đối tượng, khái niệm, cơ quan, tổ chức, vật thể, môn học, lĩnh vực có trong câu hỏi dưới đây.
Trả về kết quả dưới dạng một đối tượng JSON với trường "entities" chứa danh sách các thực thể trích xuất được.
LƯU Ý: CHỈ TRẢ VỀ ĐỐI TƯỢNG JSON. KHÔNG GIẢI THÍCH GÌ THÊM.

#####
VÍ DỤ:

Câu hỏi: "Ai là tổng thống của Hoa Kỳ?"
Đầu ra: {{ "entities": ["tổng thống", "Hoa Kỳ"] }}

Câu hỏi: "Công ty Apple sản xuất những sản phẩm gì?"
Đầu ra: {{ "entities": ["Apple", "sản phẩm"] }}

Câu hỏi: "Hội nghị COP26 có sự tham gia của những quốc gia nào?"
Đầu ra: {{ "entities": ["COP26", "quốc gia"] }}

Câu hỏi: "Người sáng lập Microsoft là ai?"
Đầu ra: {{ "entities": ["người sáng lập", "Microsoft"] }}

#####
Hãy trích xuất từ câu hỏi: {question}
"""
)

# Function to extract entities
def extract_entities(question):
    formatted_prompt = prompt.format(question=question)
    response = llm.invoke(formatted_prompt)
    
    # Get the response content or fallback to string representation
    response_text = response.content if hasattr(response, "content") else str(response)
    
    # Clean response text by removing unnecessary parts
    cleaned_response = re.sub(r"```json|```", "", response_text).strip()
    
    try:
        # Try parsing the cleaned response as JSON
        parsed_response = json.loads(cleaned_response)
        
        # Ensure that "entities" field exists in parsed response
        if "entities" not in parsed_response:
            return {"error": "No 'entities' field found in response", "raw": cleaned_response}
        
        # Map 'entities' to 'names' field for the Entities model
        parsed_response["names"] = parsed_response.pop("entities")
        
        # Return the entities as a structured object
        return Entities(**parsed_response)
    
    except json.JSONDecodeError as e:
        return {"error": f"JSON Decode Error: {str(e)}", "raw": cleaned_response}
    except KeyError as e:
        return {"error": f"Missing key: {str(e)}", "raw": cleaned_response}
    except TypeError as e:
        return {"error": f"Type Error: {str(e)}", "raw": cleaned_response}

# Test the function with a question
question = "Lĩnh vực nghiên cứu nổi bật của thầy Lê Hồng Phương? Có phải xử lý ngôn ngữ tự nhiên không?"
entities = extract_entities(question)

# Print out the extracted entities or error
print(entities)

2025-03-19 23:18:35,657 - INFO - HTTP Request: POST http://127.0.0.1:8000/v1/chat/completions "HTTP/1.1 200 OK"


names=['thầy Lê Hồng Phương', 'xử lý ngôn ngữ tự nhiên']


In [15]:
def graph_retriever(question: str) -> str:
    result = []
    
    entities = extract_entities(question)
    print(entities)
    if isinstance(entities, dict):  # If extraction failed
        return "Có vẻ không có đủ thông tin cho câu hỏi này."
    
    for entity in entities.names:
        formatted_query = entity.replace(" ", "_")  # Convert spaces to underscores
        fuzzy_query = f"{formatted_query}*"  # Add wildcard for partial matching
        query_length = len(formatted_query)  # Compute query length in Python

        response = graph.query(
            """
CALL db.index.fulltext.queryNodes('fulltext_entity_index', $query, {limit:10}) 
YIELD node, score
WITH node, score, $query_length AS query_length, size(node.id) AS id_length
WHERE score >= 0.8 OR (toFloat(query_length) / id_length) >= 0.8
CALL {
  WITH node
  MATCH (node)-[r]->(neighbor)
  RETURN CASE 
    WHEN 'Document' IN labels(node) 
    THEN coalesce([label IN labels(node) WHERE label <> '__Entity__'][0], 'Unknown') + ": " + node.source + ' - ' + type(r) + ' -> ' + neighbor.id 
    ELSE node.id + ' - ' + type(r) + ' -> ' + neighbor.id
  END AS output
  UNION ALL
  WITH node
  MATCH (node)<-[r]-(neighbor)
  RETURN CASE 
    WHEN 'Document' IN labels(neighbor) 
    THEN coalesce([label IN labels(neighbor) WHERE label <> '__Entity__'][0], 'Unknown') + ": " + neighbor.source + ' - ' + type(r) + ' -> ' + node.id
    ELSE neighbor.id + ' - ' + type(r) + ' -> ' + node.id
  END AS output
}
RETURN output LIMIT 15
            """,
            {"query": fuzzy_query, "query_length": query_length},
        )
        
        if response:
            filtered = [el['output'].replace("_", " ") for el in response if not re.search(r"^[0-9a-f]{32}", el['output'])]
            result.extend(filtered)
    
    return "\n".join(result) if result else "Có vẻ chưa có đủ thông tin cho câu hỏi này."

In [16]:
print(graph_retriever("Cần bao nhiêu điểm IELTS?"))

2025-03-19 23:21:36,685 - INFO - HTTP Request: POST http://127.0.0.1:8000/v1/chat/completions "HTTP/1.1 200 OK"


names=['IELTS', 'điểm']
Có vẻ chưa có đủ thông tin cho câu hỏi này.


In [17]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke("Hãy tìm các văn bản có liên quan đến câu hỏi sau nhất. " + question)]
    
    final_data = f"""
{graph_data}
{"".join(vector_data)}
    """

    # Replace exact matches before returning
    final_data = final_data.replace("Document:", "Tài liệu:").replace("page_content:", "Thông tin:")

    return final_data

In [18]:
print(full_retriever("Trường có nghiên cứ về lý thuyết thuần túy không?"))

2025-03-19 23:21:39,342 - INFO - HTTP Request: POST http://127.0.0.1:8000/v1/chat/completions "HTTP/1.1 200 OK"


names=['trường', 'nghiên cứu', 'lý thuyết thuần túy']

Tối ưu hóa và tính toán khoa học - Chứa -> phương pháp giải bài toán bất đẳng thức biến phân
Tài liệu: tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md - Nhắc đến -> phương pháp giải bài toán bất đẳng thức biến phân
Tối ưu hóa và tính toán khoa học - Chứa -> thiết kế thuật toán
Tài liệu: tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md - Nhắc đến -> thiết kế thuật toán
Tối ưu hóa và tính toán khoa học - Chứa -> tính toán song song
Tài liệu: tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md - Nhắc đến -> tính toán song song
Tối ưu hóa và tính toán khoa học - Chứa -> tối ưu tổ hợp
Tài liệu: tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md - Nhắc đến -> tối ưu tổ hợp
Giảng viên - Hỗ trợ -> Nghiên cứu
Tài liệu: tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md - Nhắc đến -> Nghiên cứu
Các nghiên cứu - Có ứng dụng trong -> ngành công nghiệp
Tài liệu: tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md - Nhắc đến -> Các nghiên cứu

Thông tin: trong lý th

In [19]:
import requests

# LM Studio API settings
LM_STUDIO_URL = "http://127.0.0.1:8000/v1/chat/completions"  # Update if needed

def query_lm_studio(question):
    """
    Fetches context using full_retriever() and queries LM Studio for a response.
    """
    context = full_retriever(question)

    prompt = f"""
Bạn là trợ lý cho Trường Đại học Khoa học Tự nhiên - Đại học Quốc gia Hà Nội.
Trang Web của trường: https://hus.vnu.edu.vn/
Hãy trả lời câu hỏi của người dùng dựa trên các thông tin được cung cấp.
Hãy nghĩ từng bước, xem xét tất cả các thông tin được cung cấp để đưa ra câu trả lời.
Nghiêm cấm đưa ra thông tin sai lệch.

#####
THÔNG TIN CUNG CẤP:
{context}

#####
CÂU HỎI NGƯỜI DÙNG:
{question}
Hãy trả lời bằng tiếng Việt.
"""

    # Call LM Studio's API
    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    }
    response = requests.post(LM_STUDIO_URL, json=payload)

    print(f"\nContext:\n{context}")
    print("ANSWER" + "=" * 90 + "\n")
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.text}"

In [20]:
print(query_lm_studio("Có thông tin gì về việc học các thuật toán trong trường?"))

2025-03-19 23:21:59,393 - INFO - HTTP Request: POST http://127.0.0.1:8000/v1/chat/completions "HTTP/1.1 200 OK"


names=['trường', 'học']

Context:

Giải tích Hình học - Là một lĩnh vực -> Học thuật
Hình học Đại số - Là một lĩnh vực -> Học thuật
Tài liệu: tailieu\2025-01-25-thong-tin-nghien-cuu-mim.md - Nhắc đến -> Học thuật

Thông tin: liệu và y tế, và ứng dụng biểu diễn thưa trong xử lý ảnh. Khoa cũng có các nhà nghiên cứu trong Tối ưu hóa và Tính toán Khoa học, nghiên cứu về thiết kế thuật toán, tối ưu tổ hợp, tính toán song song, và các phương pháp giải bài toán bất đẳng thức biến phân. Tất cả các
Thông tin: phương pháp giải tích số. Bên cạnh đó, các nhà nghiên cứu trong lĩnh vực Xác suất và Thống kê Toán học tập trung vào lý thuyết xác suất, thống kê toán học, các định lý giới hạn, thiết kế thí nghiệm, phân tích dữ liệu, và ứng dụng của các mô hình thống kê trong các lĩnh vực khác
Thông tin: bài toán bất đẳng thức biến phân. Tất cả các lĩnh vực nghiên cứu này được hỗ trợ bởi một đội ngũ giảng viên giàu kinh nghiệm và các trang thiết bị hiện đại, tạo điều kiện cho sinh viên và các nhà nghiên c